In [407]:
import sympy as sp
import numpy as np

N = 3
g = sp.Symbol("g", real = True)

## Creating X coordinates

In [491]:
# Creating a dictionary to store the X coordinates
X = {}
for i in np.arange(1,5):
    if i != 4:
        for j in np.arange(1,10):
            X[str(i)+str(j)] = sp.Symbol("X^{"+str(i)+"}_"+str(j), real = True)
    else:
        for j in np.arange(1,10):
            X[str(i)+str(j)] = sp.MatrixSymbol("X^{"+str(i)+"}_"+str(j),N,N)

## Creating the $\phi$'s and $\phi^{\dagger}$'s and $\phi^{(ij)}$

In [509]:
phi = {}
phi_ = {}
phi_dag = {}
for i in np.arange(1,5):
    phi[str(i)+"1"] = 1/sp.sqrt(2) * (X[str(i)+"4"] + sp.I * X[str(i)+"5"])
    phi[str(i)+"2"] = 1/sp.sqrt(2) * (X[str(i)+"6"] + sp.I * X[str(i)+"7"])
    phi[str(i)+"3"] = 1/sp.sqrt(2) * (X[str(i)+"8"] + sp.I * X[str(i)+"9"])
    
    phi_["12"] = phi["13"] - phi["23"]
    phi_["23"] = phi["21"] - phi["31"]
    phi_[""]
    
    phi_dag[str(i)+"1"] = 1/sp.sqrt(2) * ( X[str(i)+"4"] - sp.I * X[str(i)+"5"])
    phi_dag[str(i)+"2"] = 1/sp.sqrt(2) * (X[str(i)+"6"] - sp.I * X[str(i)+"7"])
    phi_dag[str(i)+"3"] = 1/sp.sqrt(2) * (X[str(i)+"8"] - sp.I * X[str(i)+"9"])

## Creating Z coordinates

In [409]:
# Building the Z-dictionary
Z = {}
for i in np.arange(1,4):
    #Scalars here
    for j in np.arange(1,4):
        if i == j: 
            continue
        else:
            P = sp.Symbol("P^{"+str(i) + str(j) +"}")
            Q = sp.Symbol("Q^{"+str(i) + str(j) +"}")
            Z[str(i) + str(j)] = (P + sp.I * Q)
    
    #Vectors here
    store_current_4i = []
    store_current_i4 = []
    for j in np.arange(1,N+1):
        P4i = sp.Symbol("P^{"+str(4)+str(i)+"}_"+str(j), real = True)
        Q4i = sp.Symbol("Q^{"+str(4)+str(i)+"}_"+str(j), real = True)
        Pi4 = sp.Symbol("P^{"+str(i)+str(4)+"}_"+str(j), real = True)
        Qi4 = sp.Symbol("Q^{"+str(i)+str(4)+"}_"+str(j), real = True)
        store_current_4i.append([P4i + sp.I * Q4i])
        store_current_i4.append([Pi4 + sp.I * Qi4])
    Z["4"+str(i)] = sp.Matrix(store_current_4i)
    Z[str(i) + "4"] = sp.Matrix(store_current_i4).transpose()

## Creating the F variables

In [480]:
Z["41"]

Matrix([
[P^{41}_1 + I*Q^{41}_1],
[P^{41}_2 + I*Q^{41}_2],
[P^{41}_3 + I*Q^{41}_3]])

In [457]:
F = {}

#Scalars
for i in np.arange(1,4):
    for j in np.arange(1,5):
        if i == j:
            continue
        if j == 4:
            F[str(i)+str(j)] = (Z[str(i)+str(j)] * Z[str(j)+str(i)])[0] + N*sp.Symbol("c^{"+str(i)+str(j)+"}")/g**2
        else:
            F[str(i)+str(j)] = Z[str(i)+str(j)] * Z[str(j)+str(i)] + sp.Symbol("c^{"+str(i)+str(j)+"}")/g**2 
